In [8]:
from  bs4 import  BeautifulSoup 
import requests
import re
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)

In [9]:
def turn_to_url(season):
    return 'http://survivor.wikia.com/wiki/Survivor:_' + season.replace(' ','_')

def process(string):
    stripped = string.strip()
    return re.sub('[0-9]+', '', stripped)

#make sure they are in order
seasons = ['Borneo', 'The Australian Outback', 'Africa', 'Marquesas', 'Thailand', 'The Amazon', 
           'Pearl Islands', 'All-Stars', 'Vanuatu', 'Palau', 'Guatemala', 'Panama', 
           'Cook Islands', 'Fiji', 'China', 'Micronesia', 'Gabon', 'Tocantins', 'Samoa', 
           'Heroes vs. Villains', 'Nicaragua', 'Redemption Island', 'South Pacific',
           'One World', 'Philippines', 'Caramoan', 'Blood vs. Water', 'Cagayan', 'San Juan del Sur',
           'Worlds Apart', 'Cambodia', 'Kaôh Rōng', 'Millennials vs. Gen X',
           'Game Changers', 'Heroes vs. Healers vs. Hustlers', 'Ghost Island','David vs. Goliath']

In [10]:
def scrape_survivor_wiki_castaway_info():
    data = []
    snum = 0
    for season in seasons:
        snum+=1
        #print(snum)
        #if season == 'Blood vs. Water':
        #print(season)
        card = requests.get(turn_to_url(season))
        soup = BeautifulSoup(card.content, 'lxml')
        summary_table = soup.select('table.wikitable.article-table')[0]
        skip = 2
        if season == 'Blood vs. Water':
            skip = 3
        lastdat = -2
        if season =='Redemption Island':
            lastdat = -3
        num_contestants = len(summary_table.select('tr'))-skip
        if season in ['Redemption Island','Pearl Islands','South Pacific']:
            ncontest = num_contestants - 2
        elif season =='Blood vs. Water':
            ncontest = num_contestants - 3
        else:
            ncontest = num_contestants
            
        for i in range(num_contestants):
            #print (i+skip)
            if len(summary_table.select('tr')[i+skip].select('td')) in [1, 2]:
                continue
            row_i = summary_table.select('tr')[i+skip]
            name = row_i.select('td b a')[0].text  
            #print(name)
            votes = row_i.select('td')[-1].text
            ageloc = row_i.select('td > small')[0].text
            daysplacing = row_i.select('td')[lastdat].text
            data.append({'season': season, 'name': name, 'ageloc': ageloc, 'daysplacing': daysplacing, 'season_number':snum,'players_in_season':ncontest,'votes_against':votes})

    rr = pd.DataFrame(data)
    
    return rr

rt = scrape_survivor_wiki_castaway_info() 

In [11]:
rr = rt.copy(deep=True)
rr

,ageloc,daysplacing,name,players_in_season,season,season_number,votes_against
0,"63, Walnut Creek, CAMusician",1st Voted OutDay 3\n,Sonja Christopher,16,Borneo,1,4\n
1,"64, Mission Hills, KSReal Estate Developer",2nd Voted OutDay 6\n,B.B. Andersen,16,Borneo,1,6\n
2,"27, San Francisco, CAAttorney",3rd Voted OutDay 9\n,Stacey Stillman,16,Borneo,1,6\n
3,"29, Edison, NJBiochemist",4th Voted OutDay 12\n,Ramona Gray,16,Borneo,1,6\n
4,"23, Spring Green, WIDairy Farmer",5th Voted OutDay 15\n,Dirk Been,16,Borneo,1,4\n
5,"27, Sherwood, ARHealth Club Consultant",6th Voted OutDay 18\n,Joel Klug,16,Borneo,1,4\n
6,"38, Clarksville, TNTeacher",7th Voted OutDay 21\n,Gretchen Cordy,16,Borneo,1,4\n
7,"24, Gold Hill, COIvy League Graduate",8th Voted Out1st Jury MemberDay 24\n,Greg Buis,16,Borneo,1,6\n
8,"22, Franklin, NHStudent",9th Voted Out2nd Jury MemberDay 27\n,Jenna Lewis,16,Borneo,1,11\n
9,"30, Willingboro, NJYMCA Basketboall Coach",10th Voted Out3rd Jury MemberDay 30\n,Gervase Peterson,16,Borneo,1,6\n


In [12]:
def add_gender(rv):
    #each row is a season that starts with the winner and goes down
    #make sure it is sorted by id
    rv['gender'] = [1,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,
                   0,1,1,0,1,0,1,0,0,1,1,0,1,0,1,0,
                   1,0,1,1,0,0,1,1,0,1,0,1,0,1,0,0,
                   0,0,0,1,1,1,0,0,1,1,0,1,0,1,0,1,
                   1,1,0,0,1,1,0,1,0,0,1,0,0,1,0,1,
                   0,1,1,1,0,0,1,0,1,1,0,0,0,1,0,1,
                   0,0,1,0,1,0,0,1,1,1,1,1,0,0,1,0,
                   0,1,0,1,1,0,0,0,1,0,1,1,0,1,1,0,1,0,
                   1,0,0,0,0,0,0,1,1,1,1,0,1,1,0,1,0,1,
                   1,0,1,0,0,1,0,0,1,1,1,1,0,1,0,1,0,0,0,1,
                   0,0,1,0,0,1,1,1,1,1,0,1,0,1,0,0,0,1,
                   1,0,1,0,1,0,1,0,1,1,1,1,0,0,0,0,
                   1,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,1,0,1,1,
                   1,1,0,1,1,0,1,1,1,0,0,1,1,0,0,1,0,0,0,
                   1,0,0,0,0,1,1,1,1,0,0,1,1,0,0,1,
                   0,0,0,0,1,0,1,1,1,0,0,0,0,1,1,1,1,1,0,1,
                   1,0,0,1,1,0,0,1,1,1,1,1,0,1,0,0,0,0,
                   1,1,0,0,1,0,0,1,1,1,0,1,0,1,0,0,
                   0,1,1,1,1,0,0,1,1,0,0,1,0,1,0,0,1,0,1,0,
                   0,0,1,0,1,1,0,0,0,1,0,1,1,1,1,1,0,1,0,0,
                   1,1,1,0,1,0,1,0,0,0,1,0,0,0,0,1,1,1,1,0,
                   1,1,0,0,0,1,1,1,1,1,0,1,0,0,0,0,1,0,
                   0,1,1,1,1,1,0,1,0,0,1,1,0,0,0,0,1,0,
                   0,0,0,0,0,1,0,1,1,1,1,1,1,0,1,1,0,0,
                   0,1,0,1,0,1,1,1,1,1,0,0,0,0,1,0,0,1,
                   1,0,0,1,1,0,0,1,1,1,1,0,0,1,1,0,1,0,0,0,
                   1,0,1,0,0,0,1,0,1,1,1,0,1,0,1,0,0,0,1,1,
                   1,1,0,1,0,0,0,1,1,0,0,0,0,1,0,1,1,1,
                   0,0,0,1,0,1,1,1,1,1,1,0,1,0,1,1,0,0,
                   1,1,0,1,0,1,1,0,0,1,0,0,1,1,0,0,1,0,
                   1,0,1,0,1,0,0,1,1,0,0,1,0,1,1,0,1,0,0,1,
                   0,0,1,0,1,1,0,1,0,1,1,1,0,0,1,0,0,1,
                   1,1,0,1,1,1,0,1,1,0,1,1,0,0,0,0,0,1,0,0,
                   0,1,1,1,0,0,0,0,0,1,0,1,0,1,0,1,1,1,1,0,
                   1,0,1,1,1,0,0,1,1,1,0,0,0,0,1,1,0,0,
                   1,1,0,0,1,1,0,0,1,0,0,0,1,1,1,0,1,0,1,0,
                   1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0,0,1,0,1]
    
    
    rv['gender'] = rv['gender'].map(lambda x: 'Male' if x == 1 else 'Female')
    
    return rv

In [13]:
def reformat_and_export_data(rv):
    def checkplace(row):
        x = row['daysplacing']
        last = ['Runner-Up', 'Sole Survivor', 'Runners-Up']
        if 'Sole Survivor' in x:
            return 1
        elif 'Second' in x:
            return 3
        elif 'Runner' in x:
            return 2
        else:
            z = (''.join(list(filter(lambda y: y.isdigit(), str(x[:3])))))
            if z == '':
                return 0
            else:
                return row['players_in_season']+1-int(z)
        
        
    def checkdays(row):
        x = row['daysplacing']
        last = ['Runner-Up', 'Sole Survivor', 'Runners-Up']
        for censored in last:
            if censored in x:
                if row['season'] == 'The Australian Outback':
                    return 42
                else:
                    return 39
        else:
            return x[x.find('Day')+4:x.find('Day')+6].strip()
        
        
    def createid(row):
        ssn = str(row['season_number']).zfill(2)
        plc = str(row['place']).zfill(2)
        
        return ssn+plc
        
        #str(rv['season_number']).zfill(2)# + str(rv['place']).zfill(2)
    
        
        
    rv['time_playing'] = rv.groupby(['name']).cumcount()+1
    rv['total_times_played'] = rv.groupby(['name'])['time_playing'].transform('max')
    rv['age'] = rv['ageloc'].str.split(',').map(lambda x: x[0])
    rv['state'] = rv['ageloc'].str.split(',').map(lambda x: x[2][:3])
    rv['days_survived'] = rv.apply(checkdays, axis=1)
    rv['place'] = rv.apply(checkplace, axis=1)
    rv['stripped'] = rv['daysplacing'].str.split('\n')
    rv['is_censored'] = ~rv['daysplacing'].str.contains('Voted Out')
    rv['is_censored'] = rv['is_censored'].astype(int)
    rv = rv[['name', 'season', 'age', 'state', 'days_survived','is_censored','place','players_in_season'
             ,'time_playing','total_times_played','season_number','votes_against']]
        
    
    rv['state'] = rv['state'].str.strip()
    rv['state'] = rv['state'].map(lambda x: 'DC' if x == 'D.' else x)

    #ADJUSTMENTS    
    rv.iloc[60,6] = 4    
    rv.iloc[107,4] = 36 # Burton PI
    rv.iloc[110,4] = 39
    rv.iloc[110,5] = 1 # Lil PI
    rv.iloc[314,4] = 8 #ben samoa

    # redemption
    rv.iloc[371:389,6] = [18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[382,4] = 21
    rv.iloc[382,5] = 1
    rv.iloc[384,4] = 37
    rv.iloc[385,4] = 38
    rv.iloc[385,5] = 0
    rv.iloc[386,4] = 39
    rv.iloc[387,4] = 39
    rv.iloc[388,4] = 39

    #south pacific
    rv.iloc[389:407,6] = [18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[389:404,4] = [3,8,11,14,16,5,24,21,27,27,30,32,35,37,38]
    rv.iloc[389:404,5] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]

    #blood vs. water
    rv.iloc[463:483,6] = [20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[463:480,4] = [1,7,6,3,1,10,16,8,18,21,22,29,32,35,26,37,28]
    rv.iloc[465:480,5] = [0,0,0,0,0,0,1,0,0,0,1,0,1,0,1]

    #dreamz 
    rv.iloc[239,4] = 39
    #michael skupin
    rv.iloc[441,4] = 39
    #sheri
    rv.iloc[461,4] = 39
    #will sims
    rv.iloc[535,4] = 39
    #tasha
    rv.iloc[555,4] = 39
    #ken
    rv.iloc[593,4] = 39
    #mike
    rv.iloc[671,4] = 39
    
    #placement for evacuations and quits
    rv.iloc[16:32,6] = [16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[186:202,6] = [16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[257:277,6] = [20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[295:311,6] = [16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[311:331,6] = [20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[407:425,6] = [18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[443:463,6] = [20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[537:557,6] = [20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[557:575,6] = [18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[653:673,6] = [20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[96:112,6] = [16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[557:575,6] = [18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[595:615,6] = [20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[112:130,6] = [18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[148:168,6] = [20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[222:241,6] = [19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[351:371,6] = [20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[425:443,6] = [18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[483:501,6] = [18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[501:519,6] = [18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[519:537,6] = [18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
    rv.iloc[575:595,6] = [20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]


    #firemaking final four if not already corrected
    rv.iloc[629,6] = 4
    rv.iloc[649,6] = 4

    rv['place_percent'] = 1-((rv['place']-1)/rv['players_in_season'])
    #rv['avg_place'] = rv.groupby(['name']).place.transform('mean')
    #rv['avg_place_percent'] = rv.groupby(['name']).place_percent.transform('mean')
    rv['days_survived'] = rv['days_survived'].astype(int)
    rv['votes_against'] = rv['votes_against'].astype(int)
    #rv['avg_days_survived'] = rv.groupby(['name']).days_survived.transform('mean')
    #rv['avg_votes_against'] = rv.groupby(['name']).votes_against.transform('mean')
    rv['id'] = rv.apply(createid, axis=1)
    
    ids = rv[['name','season_number','id']]
    ids.to_csv('/home/analytics/Survivor/ids.csv', index=True)

    rv.rename(index=str, columns={'id':'ID'},inplace=True)
    rv.set_index('ID',inplace=True)     
    rv.sort_index(inplace=True)
    
    data = add_gender(rv)
    export = data[['name', 'age', 'gender', 'state', 'season','place', 
                   'days_survived','votes_against','place_percent',#'avg_place','avg_days_survived',
                   #'avg_place_percent','avg_votes_against',
                   'time_playing','total_times_played',
                   'players_in_season','season_number','is_censored']]

    export.sort_values(['season_number','place'], ascending=[True, False],inplace=True)    
    export.to_csv('/home/analytics/Survivor/castaways.csv', index=True)
    
    return export

In [14]:
data = reformat_and_export_data(rr)
data

/home/analytics/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/analytics/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/analytics/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pan

,name,age,gender,state,season,place,days_survived,votes_against,place_percent,time_playing,total_times_played,players_in_season,season_number,is_censored
ID,,,,,,,,,,,,,,
0116,Sonja Christopher,63,Female,CA,Borneo,16,3,4,0.062500,1,1,16,1,0
0115,B.B. Andersen,64,Male,KS,Borneo,15,6,6,0.125000,1,1,16,1,0
0114,Stacey Stillman,27,Female,CA,Borneo,14,9,6,0.187500,1,1,16,1,0
0113,Ramona Gray,29,Female,NJ,Borneo,13,12,6,0.250000,1,1,16,1,0
0112,Dirk Been,23,Male,WI,Borneo,12,15,4,0.312500,1,1,16,1,0
0111,Joel Klug,27,Male,AR,Borneo,11,18,4,0.375000,1,1,16,1,0
0110,Gretchen Cordy,38,Female,TN,Borneo,10,21,4,0.437500,1,1,16,1,0
0109,Greg Buis,24,Male,CO,Borneo,9,24,6,0.500000,1,1,16,1,0
0108,Jenna Lewis,22,Female,NH,Borneo,8,27,11,0.562500,1,2,16,1,0
